## The effect of agriculture on the population of breeding birds in the Netherlands


### Introduction

The biodiversity of our planet is very essential for all the living organisms on earth. Unfortunately, the biodiversity has changed fast in the last decades. This change is caused mainly by human activities like agriculture. Agriculture can be harmful for ecosystems and can cause the loss of biodiversity. Even in the Netherlands there has been a loss of biodiversity during the last decades because of agriculture.

This research focusses on the breeding bird population in the Netherlands. Birds are very important and are part of the biodiversity. They play an important role in the food chain and in the pollution of plants.

Sources:

National Academy of Sciences. 2021. The Challenge of Feeding the World Sustainably: Summary of the US-UK Scientific Forum on Sustainable Agriculture. Washington, DC: The National Academies Press. https://doi.org/10.17226/26007.

https://www.pbl.nl/sites/default/files/downloads/HaltingBiodiversityLossInTheNetherlands_0.pdf


### Research question

What is the effect of agriculture on the population of breeding birds in the Netherlands?

### The data

Two datasets were used during this research. Both came from a different source.

The agriculture data came from Centraal Bureau voor Statestiek (CBS)
https://opendata.cbs.nl/statline/portal.html?_la=nl&_catalog=CBS&tableId=80781ned&_theme=302

The bird population data came from sovon
https://sovon.nl/indexen-en-aantallen




### Imports

In [ ]:
import yaml
import pandas as pd
from bokeh.plotting import figure, show, ColumnDataSource
import numpy as np
import re
import panel as pn
from bokeh.io import output_file, show
from bokeh.models import LinearAxis, Range1d, FactorRange, HoverTool, ColumnDataSource, Label, LabelSet
import English_column_names as ecn
from bokeh.transform import dodge
from bokeh.sampledata.sprint import sprint
from bokeh.embed import components
from bokeh.plotting import figure, show
from scipy import stats
from math import pi
from bokeh.models import BasicTicker, ColorBar, LinearColorMapper, PrintfTickFormatter
import Bird_references as br
import seaborn as sns
import matplotlib.pyplot as plt

### Creating dataframes from the files

In [ ]:
def opening_file(file_name):
    """
    Takes in a file_name and uses a configuration file to open a csv
    file as a pandas dataframe. Returns the dataframe.
    """
    with open("config.yaml", "r") as config_reader:
        files = yaml.safe_load(config_reader)
        dataframe = pd.read_csv(files[file_name], sep="\t")
    return dataframe


def form_dataframe(file_name, start_index, end_index, is_birdfile):
    """
    Takes in a file_name, a start index, an end index and a boolean.
    The file will be opened as a pandas dataframe using opening_file().
    This dataframe will be sliced using the start and end index. 
    The dataframe will be formed in certain way depending on the boolean.
    """

    # Renaming the column so it is easier to use
    column_name = [col for col in file_name][0]
    file_name.rename(columns={column_name: 'data_column'}, inplace=True)

    # Adding the column names of the agriculture dataframe on top of the dataframe. The column names will then spi
    if is_birdfile == False:
        top_row = pd.DataFrame({"data_column": column_name}, index=[0])
        file_name = pd.concat([top_row, file_name]).reset_index(drop=True)

    # Creating the dataframe
    df_x = file_name.loc[start_index:end_index].copy()
    df_x = df_x.data_column.str.split(";", expand=True)

    # If the file is not the bird file, the first and last row are sliced of
    if is_birdfile == False:
        for column in df_x:
            df_x[column] = df_x[column].str[1:-1]

    # The column names are renamed to the original column names
    df_x.columns = df_x.iloc[0]

    # The first row is sliced of, because these were the column names
    df_x = df_x.iloc[1:]

    return df_x

### Reforming the bird dataframe

The bird dataframe contains data from 29 bird species. The dataframe is reformed in two ways for different plotting purposes.

Source for regex: #https://stackoverflow.com/questions/13445241/replacing-blank-values-white-space-with-nan-in-pandas


In [ ]:
def reform_bird_df(bird_df, plot="line"):
    """
    A function that takes in a bird dataframe and a plot type.
    The function reformes the dataframe in a certain way for
    plotting purposes. The reformed dataframe is returned.
    """

    # Creating the dataframe with the usefull lines of the bird file
    bird_df = form_dataframe(bird_df, 1, 199, True)

    # Columnnames are translated to english
    bird_df = bird_df.rename(columns={
                             "English name": 'Species',
                             "Wetenschappelijke naam": "Scientific_name",
                             "Provincie": "Region"
                              })

    # Missing values are replaced with Nan
    bird_df = bird_df.replace(r'^\s*$', np.nan, regex=True)
    # The dataframe is formed based on the plotting purpose
    if plot == "line":
        # Converting all the different year columns to one column
        bird_df = bird_df.melt(id_vars=["nr.", "Soort", "Scientific_name",
                                        "Species", "Region", "Trend 1990-2020", 
                                        "Percentage jaarlijkse verandering vanaf 1990",
                                        "Trend 2009-2020",
                                        "Percentage jaarlijkse verandering vanaf 2009"],
                                        var_name="Date",
                                        value_name="Value"
                                        )

        # Dropping columns to make the data_frame clearer
        bird_df.drop(bird_df.columns[[0, 1, 5, 6, 7, 8]], axis=1, inplace=True)

        # Selecting the timeframe 2000 till 2020
        bird_df["Date"] = bird_df["Date"].astype(int)
        bird_df = bird_df.loc[bird_df["Date"].isin(
            [year for year in range(2000, 2021)])]

        # Filling the blank values with Nan
        bird_df["Value"] = bird_df["Value"].astype("float")

    # The dataframe is formed based on the plotting purpose
    elif plot == "bar":
        # Selecting the timeframe 2000 till 2020
        bird_df = bird_df.drop([str(year)
                                for year in range(1990, 2000)], axis=1)

        # Dropping columns to make the data_frame clearer
        bird_df.drop(
            bird_df.columns[[0, -4, -3, -2, -1]], axis=1, inplace=True)

    else:
        raise Exception("Wrong plot selected")

    return bird_df


# Checking the dataframe
bird_line_df = reform_bird_df(opening_file("bird_file"), "line")
bird_line_df.head()

### Checking dataframe

In [ ]:
bird_line_df.info()

In [ ]:
# Checking the dataframe
bird_bar_df = reform_bird_df(opening_file("bird_file"), "bar")
bird_bar_df.head()

### Checking dataframe

In [ ]:
bird_bar_df.info()

### Interpolation of bird data

For the barplot, the data had to be interpolated horizontally. For this dataframe, I choose the nearest method. Because the data collection of the population of some breeding birds started later than 2000. It is hard to say what the values were before the start of the data collection. Therefore, the data from the start of the data collection is used to extrapolate the missing data.

Source for extrapolation: 
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.interpolate.html
https://stackoverflow.com/questions/22491628/extrapolate-values-in-pandas-dataframe 

In [ ]:
def interpolation_bird_df(bird_df):
    """
    Function that takes in a dataframe and interpolates
    the missing data with the nearest methods at axis = 1.
    """
    # Creating a column list of all the years and a column
    # list converted to floats
    column_list = [column for column in bird_df][4:]
    float_columns = [float(column) for column in column_list]

    # Splitting the df in two, so one part can be interpolated
    interpolated_df = bird_df[column_list]
    info_df = bird_df.loc[:, "Soort": "Region"]

    # Setting the data to floats
    interpolated_df = interpolated_df.astype("float")

    # Renaming all the columns to floats
    for column in column_list:
        interpolated_df = interpolated_df.rename(
            columns={column: float(column)})

    # Interpolate using the nearest method
    interpolated_df.interpolate(method="nearest", axis=1, inplace=True)

    # Using ffill and bfill to extrapolate the Nan values
    interpolated_df = interpolated_df.ffill(axis=1).bfill(axis=1)

    # Concatenating the dfs back togheter and return them
    return pd.concat([info_df, interpolated_df], axis=1)


# Checking the interpolated data
interpolated_data = interpolation_bird_df(bird_bar_df)
interpolated_data.head()

### Reforming the agriculture dataframe

The agriculture dataframe contains data from 75 different subjects. For every subject the data frame contains counting data and the amount of companies related to the subject. During this research, the amount of companies of the subject is not used. The amount of companies per does not represent the size of agriculture in the netherlands, because the amount of companies have decreased in the Netherlands, but the remaining companies have increased in size. https://www.cbs.nl/nl-nl/nieuws/2017/09/sterke-schaalvergroting-in-de-landbouw-sinds-1950 

Source for deeptranslator: https://pypi.org/project/deep-translator/

Source reshaping df after pivot tabel: https://stackoverflow.com/questions/38951345/how-to-get-rid-of-multilevel-index-after-using-pivot-table-pandas

In [ ]:
# All the column names from the agriculture dataframe were
# translated using deeptranslator. I was not able to use this
# translator in the enviroment of the hanze, so therefore I did this 
# at home and printed the english translations and saved them in the
# module English_column_names to make thing easier

# This is the code I used:
# from deep_translator import GoogleTranslator
# english_list = [GoogleTranslator('nl', 'en').translate(text = column_name) for column_name in column_name_list]


In [ ]:
def reform_agriculture(agriculture_file, plot="line"):
    """
    A function that takes in a agriculture dataframe and a plot type.
    The function reformes the dataframe in a certain way for
    plotting purposes. The reformed dataframe is returned.
    """

    # A dictionairy for all the region codes
    region_dictionairy = {"PV20  ": "Groningen",
                          "PV21  ": "Friesland",
                          "PV22  ": "Drenthe",
                          "PV23  ": "Overijssel",
                          "PV24  ": "Flevoland",
                          "PV25  ": "Gelderland",
                          "PV26  ": "Utrecht",
                          "PV27  ": "Noord-Holland",
                          "PV28  ": "Zuid-Holland",
                          "PV29  ": "Zeeland",
                          "PV30  ": "Noord-Brabant",
                          "PV31  ": "Limburg"
                          }

    agriculture_df = form_dataframe(agriculture_file, 0, 278, False)

    # Translating the column names to english. I used a
    # translator because of column amounts
    column_name_list = []
    for column_name in agriculture_df:
        column_name = re.sub(r"_\d|\d+|S\Z", "", column_name)
        column_name = re.sub(r"(\w)([A-Z])", r"\1 \2", column_name)
        if column_name in column_name_list:
            column_name = "Remove"
        # This is the list I used to translate to english
        column_name_list.append(column_name)

    english_list = ecn.english_agriculture_column_names()

    # Creating a dictionairy with the dutch column as key
    # and the english name as value
    zipped_tuples = zip([column for column in agriculture_df], english_list)
    agriculture_df = agriculture_df.rename(
        columns={nl_name: en_name for nl_name, en_name in zipped_tuples})

    # Removing company columns
    agriculture_df.pop("Remove")

    # Selecting the timeframe 2000 till 2020
    agriculture_df = agriculture_df.rename(columns={"Periods": "Date"})
    agriculture_df["Date"] = agriculture_df["Date"].str[0:4].astype(int)
    agriculture_df = agriculture_df.loc[agriculture_df["Date"].isin(
        [year for year in range(2000, 2021)])]

    # Replacing the regioncode with the region name
    agriculture_df.replace({"Region": region_dictionairy}, inplace=True)

    # Replacing the blank values with Nan
    agriculture_df = agriculture_df.replace("       .", np.nan)

    # Turning the values into integers
    for column in agriculture_df:
        if column != "Region":
            agriculture_df[column] = agriculture_df[column].astype("float")

    if plot == "bar":

        # Reforming the dataframe by making columns per year and agriculture subject as rows
        list_of_agriculture = []
        column_list = [agriculture for agriculture in agriculture_df]
        for column in column_list[3:]:
            df_per_subject = agriculture_df.pivot_table(
                index=['Region'], columns='Date', values=column)
            df_per_subject = df_per_subject.rename_axis(None)
            df_per_subject = df_per_subject.rename_axis(
                None).rename_axis("Region", axis=1)
            df_per_subject["Agriculture"] = [column] * 12
            list_of_agriculture.append(df_per_subject)

        agriculture_df = pd.concat(list_of_agriculture)

    return agriculture_df


# Checking the dataframe
agriculture_line_df = reform_agriculture(
    opening_file("agriculture_file"), "line")
agriculture_line_df.head()

In [ ]:
# Checking the dataframe
agriculture_bar_df = reform_agriculture(
    opening_file("agriculture_file"), "bar")
agriculture_bar_df.head()

### Checking dataframe

In [ ]:
agriculture_line_df.info()

### Data quantity

There is data from a lot of different agricultural practices and different bird species. But there is not a lot of data of all the species and agricultural practices per year. The data was collected from 12 different regions in the Netherlands. So, every year there were a maximum 12 values per bird species and agricultural practice.

In [ ]:
# List of all birds
bird_list = bird_line_df["Species"].unique()
agriculture_list = [col for col in agriculture_line_df][3:]

print("There is data available:\n" +
      f"   {len(bird_list)} bird species\n" +
      f"   {len(agriculture_list)} types of agriculture\n" +
      "   12 regions of the Netherlands\n" +
      "   From 2000-2020 from both datasets (21 years total)\n" +
      f"   {len(bird_list) * 12 * 21 * len(agriculture_list)} data points in total"
      )

print("\nPer bird species data available:\n" +
      f"   {len(agriculture_list)*12*21} data points in total\n" +
      f"   {12 * 21} data points per agriculture\n" +
      f"   {21 * len(agriculture_list)} data points per region\n" +
      f"   {len(agriculture_list)*12} data points per year\n" +
      f"   21 data points per region and agriculture"
      )

### Data quality
There was a lot of data missing in the data frames.  Some of the data of the bird data frame was missing because the data collection started later than the year 2000.  There was also data missing from the agriculture data frame. Several practices had missing data from different years and different regions.

In [ ]:
total_data = len(bird_list) * 12 * 21 * len(agriculture_list)
missing_bird_data = bird_line_df.isna().sum().sum()
missing_agriculture_data = agriculture_line_df.isna().sum().sum()

print("The amount of missing data:\n" +
      f"{missing_bird_data} missing bird points ({missing_bird_data/(len(bird_list) * 12 * 21)*100}%)\n" +
      f"{missing_agriculture_data} missing agriculture data points ({missing_agriculture_data/(total_data/len(bird_list))*100}%)\n"
      )

### Merging the two dataset togheter per bird species for the lineplot
To create an interactive lineplot, I first merged the bird dataframe and the agriculture dataframe per bird species. A dictionairy with the bird as key and the merged dataframe as value was created. This dictionairy will be used for the lineplot.

In [ ]:
def create_df_dict(bird_df, agriculture_df):
    """
    Takes in a bird data frame and a agriculture data frame.
    The dataframes are be merged per bird species. A dictionairy
    is created with the bird as key and the merged dataframe as value.
    """

    # A list of all the different bird species
    bird_list = bird_df["Species"].unique()

    # A dictionairy with the bird as key and the corresponding
    # data from the bird dataframe
    bird_df_dict = {
        bird: bird_df.loc[bird_df["Species"] == bird] for bird in bird_list}

    # A dictionairy with the bird as key and the corresponding data
    # from the bird dataframe merged with the agriculture dataframe
    df_dict = {bird: (agriculture_df.merge(bird_df_dict[bird], on=[
                      "Date", "Region"], how='left')) for bird in bird_df_dict}

    return df_dict


# Checking if the function works
df_dict = create_df_dict(bird_line_df, agriculture_line_df)
df_dict["Greater Canada Goose"].head()

### Concatenating the dataframes togheter for the bar plot
For the barplot, the dataframes were also concatenated. To normalize the data, the percentage of change that occured between the start year and end year was calculated. This percentage was added to the merged dataframe as a column.

In [ ]:
def change_percentage_column(file_name, region="Groningen", start_year=2000, end_year=2020, how="concat"):
    """
    A function that takes in a file_name, a region, a start_year and a end_year.
    Based on these parameter this function creates a dataframe with data of the given years and region.
    Then the change percentage between these years are calculated. 
    The change precentage values are added as a column to the dataframe. The dataframe is then returned.
    """

    # The bird dataframe
    if file_name == "bird_file":
        df_x = interpolation_bird_df(
            reform_bird_df(opening_file("bird_file"), "bar"))
        if region != "All":
            # Only data of the chosen region is selected
            df_x = df_x.loc[df_x["Region"] == region]
        column = "Species"
        # For the color of the bars in the barplot a color column is added
        df_x["Color"] = ["lightseagreen"] * len(df_x)

    # The agriculture dataframe
    elif file_name == "agriculture_file":
        if region != "All":
            # Only data of the chosen region is selected
            df_x = reform_agriculture(opening_file(
                "agriculture_file"), "bar").loc[region]
        else:
            df_x = reform_agriculture(opening_file("agriculture_file"), "bar")
        column = "Agriculture"
        # For the color of the bars in the barplot a color column is added
        df_x["Color"] = ["sandybrown"] * len(df_x)
        df_x = df_x.reset_index(level=0).rename(columns={"index": "Region"})

    else:
        raise Exception("Wrong file_name selected")

    # Only the neccesary columns are kept in the dataframe
    df_x = df_x[[column, "Color", "Region", start_year, end_year]]

    # The change percentage is calculated and added as column.
    # This percentage is used to normalize the data
    df_x["Change"] = df_x[end_year] - df_x[start_year]
    df_x["Percentage"] = df_x["Change"]/df_x[start_year]

    if how == "merge":
        df_x.drop(columns=[start_year, end_year,
                           "Color", "Change"], inplace=True)

    return df_x


def merging_barplot_data(bird_df, agriculture_df, how="concat"):
    """
    Takes in two dataframes, where the changepercentage is calculated between two years.
    It merges the two dataframes togheter and returns the merged dataframe.
    """
    # Renaming some columns to label, so the columns can be concatenated
    bird_df.rename(columns={"Species": "Label"}, inplace=True)
    agriculture_df.rename(columns={"Agriculture": "Label"}, inplace=True)

    if how == "concat":
        # Concatenating and returning the data
        return pd.concat([agriculture_df, bird_df])

    elif how == "merge":
        return pd.merge(agriculture_df, bird_df)


# Testing if the functions work
bird_df = change_percentage_column("bird_file", region="Groningen")

agriculture_df = change_percentage_column("agriculture_file", region="All")

percentage_df = merging_barplot_data(bird_df, agriculture_df)
agriculture_df.head()

### Testing the data for the line plot

In [ ]:
# Checking if the dataframes are the same length (when only one
# bird species is selected from the dataframe)
print("The bird df has {} rows when one species is selected".format(
    len(bird_line_df.loc[bird_line_df["Species"] == "Grey Heron"])))
print("The agriculture df has {} rows".format(
    len(agriculture_line_df["Date"])))
print("The df of Greater Canada Goose has {} rows".format(
    len(df_dict["Greater Canada Goose"]["Date"])))

### Line plot

For the visualization of the data per bird species, per agriculture subject, and per region, an interactive lineplot is created. This lineplot shows if the bird population or agriculture has been changed between 2000 and 2020. This plot shows specificly data from one bird species with one agriculture from one region in the Netherlands. This way it is easy to compare to two things on this specific level. But this plot does not give answer to the research question. The data needs to be visualized more broadly before the question can be answered.

Source extra y range: https://docs.bokeh.org/en/1.0.0/docs/user_guide/examples/plotting_twin_axes.html

In [ ]:
# A dictionairy with bird names as keys and scientific names as values
name_dict = {bird: scientific for bird, scientific in zip(
    bird_line_df["Species"].unique(), bird_line_df["Scientific_name"].unique())}


def make_plot(bird="Greater Canada Goose", region="Groningen", agriculture="Land use Total"):
    """
    Takes in a bird, region and agriculture. Creates a plot with the data from the bird data and agriculture data.
    Only uses the data of the given bird and agriculture from the given region.
    """

    # Uses dataframe with only data from the selected region and bird
    df = df_dict[bird].loc[df_dict[bird]["Region"] == region].interpolate()

    # A range for the y axis
    agriculture_start_range = df[agriculture].min() * 0.95
    agriculture_end_range = df[agriculture].max() * 1.05

    # A range for the second y axis
    bird_start_range = df["Value"].min() * 0.95
    bird_end_range = df["Value"].max() * 1.05

    # Making a plot
    p = figure(title=f"{bird} ({name_dict[bird]})", y_axis_label="Bird indexes",
               x_axis_label="Time in years", y_range=(bird_start_range, bird_end_range))
    p.line(df["Date"], df["Value"], color="lightseagreen",
           line_width=5, legend_label="Birds")
    p.extra_y_ranges = {"y_range": Range1d(
        start=agriculture_start_range, end=agriculture_end_range)}

    # Making a line for the plot
    p.line(df["Date"], df[agriculture], color="sandybrown",
           line_width=5, legend_label="Agriculture", y_range_name="y_range")
    p.add_layout(LinearAxis(y_range_name="y_range"), 'right')

    return p


# Making lists for the widgets
bird_list = [bird for bird in bird_line_df["Species"].unique()]
region_list = [region for region in bird_line_df["Region"].unique()]
agriculture_list = [column_name for column_name in agriculture_line_df][3:]


# Testing if the interactive plot works
pn.extension()
pn.interact(make_plot, bird=bird_list, region=region_list,
            agriculture=agriculture_list)

### Testing the data for the bar plot

In [ ]:
# Checking if the dataframes have the same amount of years
print("The bird df has {} year columns".format(
    len([col for col in bird_bar_df][4:])))
print("The agriculture df has {} columns".format(
    len([col for col in agriculture_bar_df][1:])))

### Bar plot

To create a more broadened visualization, a barplot was made for all the data per region. With this barplot it is easy to see what happened to all the agriculture and bird populations between 2000 and 2020 per region. With this bar plot it can be concluded that a few bird species population such as red crested pochard has increased as lot. Some bird populations have decreased. The same can be said about the agriculture. Some agriculture has increased and some has decreased. The bar plot also shows some of the measured populations and agriculture has not changed a lot between 2000 and 2020.

This graph gives a broad idea about what happened with the bird population and agriculture between 2000 and 2020. But this still does not answer the research question. To answer the research question, it has to be determined wether there is an correlation between the change of agriculture and and the change of the bird population.

In [ ]:
def bar_plot(region="Groningen", end_year=2020, x_range=20):
    """
    Takes in a x_range, region and a end year. Creates a plot
    with the data from the bird data and agriculture data.
    Only uses the data from the given region.
    """

    # Creating the dataframe
    bird_df = change_percentage_column("bird_file", region, 2000, end_year)
    agriculture_df = change_percentage_column(
        "agriculture_file", region, 2000, end_year)
    df = merging_barplot_data(bird_df, agriculture_df)

    # The x and y values for the plot
    x = df["Percentage"]
    y = df["Label"]

    # The list of the used colors
    color_list = df["Color"]

    # Making the plot
    p = figure(y_range=FactorRange(factors=y), plot_height=1000, width=900, title="Changes between 2000 and 2020",
               toolbar_location=None, x_axis_label='Change (%)')
    # Because some data had a really big percentage value, The x_range can be adjusted
    p.x_range = Range1d(-5, x_range)

    # Horizontal bars
    p.hbar(y=y, right=x, color=color_list)

    return p


# Lists for the widgets
x_range = range(0, 330)
year_list = [year for year in range(2000, 2021)]
region_list = [region for region in bird_line_df["Region"].unique()]
region_list.append("All")


# Testing if the interactive plot works
pn.extension()
pn.interact(bar_plot, start_year=2000, end_year=year_list,
            region=region_list, x_range=x_range)

### Scatter plot

The previous plots were able to visualize what happened to the agriculture subjects and the breeding bird population, but the plots did not show a correlation between the two subjects. Therefore a scatter plot was created. The percentage of change of a chosen bird species is plotted against the percentage of change of a agriculture subject. A regression line is also created and the R squared is calculated. With this plot is easy to see per bird species and agriculture subject if there is a correlation between the two parameters. With this plot, the question could be answered. But because of the many parameters, it is still hard to see which bird species are influenced and how many bird species are influenced by agriculture.

Source linregress: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html

Source regressionline: https://stackoverflow.com/questions/54603873/bokeh-plot-regression-lines-on-scatter-plot

In [ ]:
def x_y_values(bird="Greater Canada Goose", agriculture="Land use Total", start_year=2000, end_year=2020):
    """
    Takes in a bird, agriculture and start and end year.
    Creates x and y values for the scatter plot.
    Only uses the data of the given bird and agriculture. 
    """

    # Dataframes are created with the chosen parameters
    bird_df = change_percentage_column(
        "bird_file", region="All", start_year=start_year, end_year=end_year)
    bird_df = bird_df.loc[bird_df["Species"] == bird]
    agriculture_df = change_percentage_column(
        "agriculture_file", region="All", start_year=start_year, end_year=end_year)
    agriculture_df = agriculture_df.loc[agriculture_df["Agriculture"] == agriculture]

    # To avoid errors because of the missing data, both data_frames are adjusted in such a way that if data is missing from a certain region in one dataframe, that the data in the other dataframe will also be removed
    agriculture_regions, bird_regions = agriculture_df["Region"], bird_df["Region"]
    bird_df = bird_df.loc[bird_df["Region"].isin(agriculture_regions)]
    agriculture_df = agriculture_df.loc[agriculture_df["Region"].isin(
        bird_regions)]

    return bird_df["Percentage"], agriculture_df["Percentage"]

In [ ]:
def make_scatter_plot(bird="Little Grebe", agriculture="Land use Total", start_year=2010, end_year=2020):
    """
    Takes in a bird species, a agriculture subject and a start and end year. 
    Creates a plot with the data from the bird data plotted against the agriculture data.
    """
    # Creating the x and y values using a funciton
    x, y = x_y_values(bird, agriculture, start_year=2010, end_year=2020)

    # Calculating the slope, intercept and R value using linregress
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

    # Creating the regression line
    regressionline = [slope*data + intercept for data in x]

    # Calculating R squared
    r_squared = r_value**2

    # Creating the plot
    p = figure(width=400, height=400,
               title="Linear regression scatterplot\nr-squared = {:.3f}".format(r_squared))
    p.circle(x, y, color='navy')
    p.line(x, regressionline, color='red', legend_label='y=' +
           str(round(slope, 2))+'x+'+str(round(intercept, 2)))

    return p


# Testing if the plot works
pn.extension()
pn.interact(make_scatter_plot, bird=bird_list,
            agriculture=agriculture_list, start_year=year_list, end_year=year_list)

### Heatmapping the correlation

With the scatter plot it was easy to see if the selected argiculture subject had an effect on the selected bird. But it was hard to see in one look which birds all were effected by which agriculture subjects. Therefore, a correlation heatmap was made using seaborn.

Correlation information:
   - corr= +1 indicates perfect positive correlation
   - corr = –1 indicates perfect negative correlation
   - corr = 0 indicates no correlation

source: https://medium.com/mlearning-ai/heatmap-for-correlation-matrix-confusion-matrix-extra-tips-on-machine-learning-b0377cee31c2

In [ ]:
def make_correlation_df(df_x):
    """
    Takes in a dataframe.
    Creates a dataframe with the correlation values
    between the bird data and the agriculture data.
    """

    column = [col for col in df_x][0]
    df_x = df_x.pivot_table(
        index=['Region'], columns=column, values="Percentage")
    df_x = df_x.rename_axis(None)
    df_x = df_x.rename_axis(
        None).rename_axis("Region", axis=1)

    return df_x

# Creating the dataframes for the heatmap
heat_bird_df = make_correlation_df(
    change_percentage_column("bird_file", region="All", how="merge"))
heat_agriculture_df = make_correlation_df(
    change_percentage_column("agriculture_file", region="All", how="merge"))

# Joining the dataframes togheter and converting the values to
# correlation values with .corr()
heatmap_df = heat_bird_df.join(heat_agriculture_df).corr().rename_axis(
    None).rename_axis("Region", axis=1)

# Slicing the dataframe, so only birds with agricultural activities are compared
heatmap_df = heatmap_df.loc[:, "Agriculture Total":].loc[:"White Stork"]

# Checking the dataframe
heatmap_df.head()

In [ ]:
def create_heatmap(df_x):
    """
    A function that takes in a heatmap dataframe and
    creates a heatmap from that dataframe.
    """
    p = plt.figure(figsize=(15, 15))
    sns.heatmap(heatmap_df, fmt='.2g', cmap="PiYG")
    plt.title(
        'Correlation between breeding bird population and agriculture', fontsize=20)

    return p

### Bonus

In [ ]:
def image(bird):
    """
    A function that takes in a bird and return the bird picture.
    """
    with open("config.yaml", "r") as config_reader:
        files = yaml.safe_load(config_reader)
    reference_string = br.bird_references(bird)

    # Adding references to the pictures
    reference = pn.widgets.TextInput(name='Reference', value=reference_string)
    image = pn.pane.PNG(files[bird], width=500)

    return pn.GridBox(*[reference, image], ncols=1, nrows=2)

### Combining plots
The plots are combined and mad interactive with panel using panel. In the first cell, I only combined the first four plot, because the fifth plot was created with matplotlib. The heatmap is therefore also shown below.

In [ ]:
pn.extension()
p1 = pn.interact(make_plot, bird=bird_list, region=region_list,
                 agriculture=agriculture_list)
p2 = pn.interact(make_scatter_plot, bird=bird_list,
                 agriculture=agriculture_list, start_year=year_list, end_year=year_list)
p3 = pn.interact(image, bird=bird_list)
p4 = pn.interact(bar_plot, start_year=year_list,
                 end_year=year_list, region=region_list)

tabs = pn.Tabs(('Lineplot', p1), ("Linear regression", p2),
               ("Bird picture", p3), ("Bar plot overview", p4))

tabs

In [ ]:
# Heatmap
heatmap = create_heatmap(heatmap_df)
heatmap.show()

### Discussion/Conclusion

#### Research question
The research question of this research was: What is the effect of agriculture on the population of breeding birds in the Netherlands? It was expected that agriculture did have an effect on the population of breeding birds.

#### Data quality and quantity
A lot of data was missing , and for some bird species there was no data at all from some of the regions.  Also, when comparing the two dataframes, there was an overlap of only 21 years of data. Some of the bird species had a missing data because the bird indexing started later than the year 2000. 

#### The used plots
First, a line plot was created to visualize data of the agricultural practices and the bird indexes. With this plot it is easy to see if a bird population changed together with an agricultural practice. But unfortunately was this graph not enough to have an idea about the influence of  all the different agricultural practices on the bird species. To make a more broad visualization, a bar plot was created with all the data. With this bar plot it was easy to see in one look what happened to all the bird species and agricultural practices. The percentage of change between 2000 and 2020 was used as a value to normalize the data.

All the data was visualized using the line and bar plot. But the plots did not say anything about correlation between the two datasets. That is why a scatter plot was created with the change percentage of the bird species plotted against the change percentage of the agricultural practices. In this plot a regression line and a R square was added. Using this graph it can be concluded if there is a correlation between the population change of a bird species and the change of an agricultural practice.

However, these plots did not easily show which bird species was affected by which agricultural practice. That is why a correlation heatmap was created with the agricultural practices on the y axis, the bird species on the x axis. This way it was better to determine which agricultural practice had an effect on which bird species.

#### Found correlations
The five birds with the strongest correlations:
    Common Eider, 
    Barnacle Goose, 
    Common pochard,
    Red-crested Pochard,
    Common qual

The heatmap suggests that there are some positive and negative correlations. To confirm that these correlations are true, more research is required for this subject, preferably with data from more than 12 regions. 


#### Final conclusion
It seems that some of the bird populations are influenced by some agricultural practices and the majority of the bird populations not. There is not data enough data to say that there is or is not a correlation between the two parameters. Also, other factors such as rainfall and infrastructure were not taken into account during this study.

Overall, it can be concluded that agricultural practices might have a positive and negative effect on some of the breeding bird populations, however further research is required to confirm this.

For future studies it is recommended to use data from more than only 12 regions, use data from a bigger timeframe, and to take other factors, such as rainfall and infrastructure, into account .
